IMPORT

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

TRAIN DATA:

In [2]:
# importing required modules 
from zipfile import ZipFile 


# opening the zip file in READ mode 
with ZipFile("/content/drive/My Drive/train.zip", 'r') as zip: 
	# printing all the contents of the zip file 
	zip.printdir() 

	# extracting all the files 
	print('Extracting all the files now...') 
	zip.extractall() 
	print('Done!') 

File Name                                             Modified             Size
train.csv                                      2020-03-19 00:50:22     88497206
Extracting all the files now...
Done!


In [0]:
data = pd.read_csv("/content/train.csv")

In [4]:
data.shape

(279, 60485)

Test data:

In [5]:
# importing required modules 
from zipfile import ZipFile 


# opening the zip file in READ mode 
with ZipFile("/content/drive/My Drive/test.zip", 'r') as zip: 
	# printing all the contents of the zip file 
	zip.printdir() 

	# extracting all the files 
	print('Extracting all the files now...') 
	zip.extractall() 
	print('Done!') 

File Name                                             Modified             Size
test.csv                                       2020-03-19 00:50:22     23186470
Extracting all the files now...
Done!


In [0]:
test_data = pd.read_csv("/content/test.csv")
test_data=test_data.drop(['id'], axis = 1)

PREPROCESSING:

In [0]:
labels=[]
labels=list(data["Label"])

In [8]:
data_new=data
data_new=data_new.drop(["id","Label"], axis = 1)
data_new.head()

ENSG00000242268.2  ENSG00000270112.3  ...  ENSG00000123685.7  ENSG00000181518.3
0           1.322600           0.000000  ...           1.708020                0.0
1           0.000000           0.006431  ...           0.182464                0.0
2           0.000000           0.000000  ...           0.386022                0.0
3           0.201191           0.000000  ...           0.666707                0.0
4           0.043040           0.000000  ...           0.513950                0.0

[5 rows x 60483 columns]

In [0]:
data_new=data_new.astype(np.float)
X_train=data_new

In [0]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(data_new,labels, test_size=0.1)
# print("X_train",X_train.shape)
# print("X_test",X_test.shape)
# print("y_train",len(y_train))
# print("y_test",len(y_test))
# print("test_data",test_data.shape)

NORMALIZATION:

In [0]:
# log2 normalise
X_train=np.log2(1+X_train)
test_data=np.log2(1+test_data)
# low variance normalisation

In [0]:
data_col=list(X_train.columns)
X_train_var=list(X_train.var())
for i in range(0,len(X_train_var)): 
   if X_train_var[i]<0.025:
     X_train=X_train.drop([data_col[i]],axis=1)
     test_data=test_data.drop([data_col[i]],axis=1)


In [16]:
test_data.head()

ENSG00000242268.2  ENSG00000167578.15  ...  ENSG00000231119.2  ENSG00000123685.7
0           0.000000            0.522842  ...           0.164422           0.304786
1           0.193829            0.622900  ...           0.306519           0.322804
2           0.023347            0.579418  ...           0.180143           0.069272
3           0.000000            0.434230  ...           0.437276           0.430381
4           0.436196            0.351468  ...           0.128643           0.240638

[5 rows x 23528 columns]

In [17]:
print("X_train",X_train.shape)
print("test_data",test_data.shape)

X_train (279, 23528)
test_data (71, 23528)


In [0]:
X_train_=X_train
test_data_=test_data

In [19]:
test_data_.head()

ENSG00000242268.2  ENSG00000167578.15  ...  ENSG00000231119.2  ENSG00000123685.7
0           0.000000            0.522842  ...           0.164422           0.304786
1           0.193829            0.622900  ...           0.306519           0.322804
2           0.023347            0.579418  ...           0.180143           0.069272
3           0.000000            0.434230  ...           0.437276           0.430381
4           0.436196            0.351468  ...           0.128643           0.240638

[5 rows x 23528 columns]

In [0]:
# minmax normalise:
for col in X_train_.columns:
    if (X_train_[col].max() - X_train_[col].min())!=0:
      X_train_[col] = (X_train_[col] - X_train_[col].min())/ (X_train_[col].max() - X_train_[col].min())
    if (test_data_[col].max() - test_data_[col].min())!=0:
      test_data_[col] = (test_data_[col] - test_data_[col].min())/ (test_data_[col].max() - test_data_[col].min())

FEATURE SELECTION:

In [0]:
colu_test_data=list(X_train_.columns)
S = SelectKBest(f_classif, k=100)
xkbest = S.fit_transform(X_train_,labels)
features = S.get_support(indices=True)

In [22]:
xkbest.shape

(279, 100)

In [0]:
for i in colu_test_data:
  if colu_test_data.index(i) not in features:
      test_data_=test_data_.drop([i],axis=1)

In [24]:
test_data_.shape

(71, 100)

pca

In [0]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=70)
pca.fit(xkbest)
pca_train=pca.transform(xkbest)
pca_test=pca.transform(test_data_)

In [26]:
len(labels)

279

MODEL:

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

In [94]:

# from sklearn.model_selection import cross_val_score
X = pca_train
y = labels
clf = MLPClassifier(solver='adam', alpha=1e-8,
                    hidden_layer_sizes=(49,2), random_state=1)
clf.fit(X, y)
scores = cross_val_score(clf, X, y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))



/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.64 (+/- 0.13)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [96]:
y_pred1=clf.predict(pca_test)
from sklearn.metrics import accuracy_score
y_true = [-1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1, -1,  1,  1,  1,  1,
        1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1, -1,  1, -1,
        1,  1,  1, -1,  1,  1, -1, -1,  1, -1,  1,  1, -1,  1,  1, -1,  1,
        1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1, -1, -1,
       -1, -1, -1]
accuracy_score(y_true, y_pred1)

0.4507042253521127

In [0]:
t_id= pd.read_csv("/content/test.csv")
id_=list(t_id['id'])
y_pred_=list(y_pred)
dic_={'id':id_,'Labels':y_pred_}
ans_=pd.DataFrame(dic_)
ans_.to_csv('new_output_level6_new__.csv')

SVM:

In [101]:

from sklearn.svm import SVC
set_svm =SVC(C=1, cache_size=100, gamma='scale', kernel='rbf',
      probability=True)
X = pca_train
y = labels

from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
rf=BaggingClassifier(base_estimator=set_svm,n_estimators=30, random_state=1)


rf.fit(X,y)
scores = cross_val_score(rf, X, y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.70 (+/- 0.14)


In [102]:
y_pred2=rf.predict(pca_test)
from sklearn.metrics import accuracy_score
y_true = [-1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1, -1,  1,  1,  1,  1,
        1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1, -1,  1, -1,
        1,  1,  1, -1,  1,  1, -1, -1,  1, -1,  1,  1, -1,  1,  1, -1,  1,
        1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1, -1, -1,
       -1, -1, -1]
accuracy_score(y_true, y_pred2)

0.4647887323943662

In [0]:
t_id= pd.read_csv("/content/test.csv")
id_=list(t_id['id'])
y_pred_=list(y_pred2)
dic_={'id':id_,'Label':y_pred_}
ans_=pd.DataFrame(dic_)
# ans_.to_csv('svm.csv')

In [0]:

ans_.to_csv('svm.csv')

Random forest:

In [109]:
from sklearn.ensemble import RandomForestClassifier
import math
clf = RandomForestClassifier(max_depth=30, random_state=1, max_features=int(math.sqrt(len(X))))
X = pca_train
y = labels
clf.fit(X,y)
scores = cross_val_score(clf, X, y, cv=10)
# yop=clf.predict(pca_test)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.66 (+/- 0.15)


In [110]:
y_pred3=clf.predict(pca_test)
from sklearn.metrics import accuracy_score
y_true = [-1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1, -1,  1,  1,  1,  1,
        1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1, -1,  1, -1,
        1,  1,  1, -1,  1,  1, -1, -1,  1, -1,  1,  1, -1,  1,  1, -1,  1,
        1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1, -1, -1,
       -1, -1, -1]
accuracy_score(y_true, y_pred3)

0.5633802816901409

In [0]:
t_id= pd.read_csv("/content/test.csv")
id_=list(t_id['id'])
y_pred_=list(y_pred3)
dic_={'id':id_,'Label':y_pred_}
ans_=pd.DataFrame(dic_)
ans_.to_csv('randomforest1.csv')

In [112]:
y_pred3

array([ 1,  1,  1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1,  1,  1,  1, -1,  1, -1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1,  1, -1, -1,  1, -1,
        1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1,
        1, -1, -1])